!pip install pymystem3 selenium html2text requests beautifulsoup4 atlas json2xml  

Don't forget to install **[Chrome Driver](https://sites.google.com/a/chromium.org/chromedriver/downloads)**

In [1]:
!pip install pymystem3 selenium html2text requests beautifulsoup4 atlas json2xml  

In [2]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options

from time import sleep
import tqdm
import json
import time

import requests
from bs4 import BeautifulSoup
from html2text import html2text

import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer

In [3]:
import warnings
warnings.filterwarnings('ignore')

## 1. Парсинг URL адресов с новостями (Selenium)

In [140]:
#Download driver using the link at the top of the notebook
chrome_driver_path = "chromedriver.exe"
links_filename = "links.txt"

In [141]:
categories = ["incidents","economy", "society", "science", "defense_safety"]

In [40]:
def parse_category_ria(category, scrolls_num=200, scroll_size=500):
    
    links = []
    prefs = {"profile.managed_default_content_settings.images": 2,
             "profile.default_content_settings.cookies": 2}
    
    # configuring driver
    chrome_options = Options()
    chrome_options.add_extension('adblock.crx')
    chrome_options.add_experimental_option("prefs",prefs)
    chrome_options.add_argument('--disable-application-cache')
    driver = Chrome(executable_path=chrome_driver_path, chrome_options=chrome_options)
    driver.implicitly_wait(10)
    driver.close()
    print("Current session is {}".format(driver.session_id))
    
    # configuring adblock
    driver.switch_to.window(driver.window_handles[0])
    driver.get('chrome-extension://gighmmpiobklfepjocnamgkkbiglidom/options.html')
    driver.find_element_by_id('acceptable_ads').click()#adblock customization
    
    # parse articles list
    try:
        
        driver.get('https://ria.ru/' + category + '/')
        element = driver.find_element_by_class_name('list-more')
        
        # finding the position of 'Show More' button
        driver.execute_script(f"window.scrollTo(0, {element.location['y'] - 100});")
        
        # click on 'Show More' button
        driver.find_element_by_class_name('list-more').click()
        
        # scrolling webpage to get more articles
        for i in tqdm.tqdm(range(1, scrolls_num + 1)):
            driver.execute_script(f"window.scrollTo(0, {scroll_size*i});")
            sleep(0.2)

        # retrieving articles links
        for element in tqdm.tqdm(driver.find_elements_by_class_name("list-item"),position=0):
            links.append(element.find_element_by_tag_name('a').get_attribute('href'))
            #print(element.find_element_by_tag_name('a').get_attribute('href'))
    
    finally:    
        driver.quit()
        
    return links

In [41]:
# links = parse_category_ria('defense_safety', scrolls_num=30, scroll_size=300)

In [47]:
def get_articles_links(categories, output):
    for category in categories:
        with open(output,'a') as file:
            links = parse_category_ria(category, scrolls_num=2000, scroll_size=150)
            print(category, ':' ,len(links))
            for link in links:
                file.write(json.dumps({
                    'link' :link,
                    'category': category
                },ensure_ascii=False) + '\n')

In [48]:
get_articles_links(categories, links_filename)

Current session is 44ba512f344ad5f4f5a57a8ea0172bb7


100%|██████████████████████████████████████████████████████████████████████████████| 1620/1620 [00:43<00:00, 37.25it/s]


incidents : 1620
Current session is 34d5d82a9db45bb1e223e63754e0427e


100%|██████████████████████████████████████████████████████████████████████████████| 1560/1560 [00:42<00:00, 37.11it/s]


economy : 1560
Current session is f364bd91b40d46a8677cf8dd8ebe5479


...

society : 1572
Current session is ad6c6da4d0e0bc265baa958c818f532a


100%|██████████████████████████████████████████████████████████████████████████████| 1480/1480 [00:38<00:00, 37.97it/s]


science : 1480
Current session is 2526046b18b0a598c0e78d2ae0e53b4e


100%|██████████████████████████████████████████████████████████████████████████████| 1460/1460 [00:39<00:00, 36.75it/s]


defense_safety : 1460


## 2. Парсинг новостей РИА Новости

In [144]:
posts = []
with open(links_filename) as file:
    for line in file:
        posts.append(json.loads(line))

In [146]:
for element in tqdm.tqdm(posts):
    soup = BeautifulSoup(requests.get(element['link']).text, "lxml")
    text = ''
    tags = []
    title = None
    
    try:
        title = soup.find(attrs={'class': 'article__title'}).text

        # the first 
        for paragraph in soup.find_all(attrs={'class': 'article__text'}):
            text += html2text(paragraph.text).replace('\n', ' ') + ' '

        for tag in soup.find('div', attrs={'class': 'article__tags'}).find_all('a'):
            tags.append(tag.text)
    
    #     print(title)
    #     print(text)
    #     print(tags)
        with open('ria_news.json','a') as file:
            file.write(json.dumps({
                'article_id' : element['link'],
                'title' : title,
                'category': element['category'],
                'tags' : tags,
                'text': text,
            },ensure_ascii=False) + '\n')
    except:
        print(f"Err in {element['link']}")
        pass

  4%|██▋                                                                          | 271/7692 [04:39<2:47:09,  1.35s/it]

Err in https://ria.ru/20220111/sud-1767361637.html


 12%|█████████▏                                                                   | 914/7692 [15:48<1:22:06,  1.38it/s]

Err in https://ria.ru/20211224/moshennichestvo-1765421360.html


 22%|████████████████▍                                                           | 1659/7692 [28:39<1:56:10,  1.16s/it]

...

 96%|████████████████████████████████████████████████████████████████████████▊   | 7365/7692 [2:08:30<07:01,  1.29s/it]

Err in https://ria.ru/20210909/raketa-1749276476.html


 96%|████████████████████████████████████████████████████████████████████████▊   | 7373/7692 [2:08:41<06:55,  1.30s/it]

Err in https://ria.ru/20210907/tank-1748952380.html


100%|████████████████████████████████████████████████████████████████████████████| 7692/7692 [2:15:04<00:00,  1.05s/it]


### Пример парсинга новостной статьи

In [147]:
link = 'https://ria.ru/20220116/mirotvortsy-1768069681.html'
soup = BeautifulSoup(requests.get(link).text, "lxml")

print(soup.find( attrs={'class': 'article__title'}).text)
text = ''
tags = []
for paragraph in soup.find_all(attrs={'class': 'article__text'}):
    text += html2text(paragraph.text).replace('\n', ' ') + ' '
print(text)

for tag in soup.find('div', attrs={'class': 'article__tags'}).find_all('a'):
    print(tag.text.lower())
    

Двадцатый самолет ВКС с российскими миротворцами вылетел из Казахстана
АЭРОДРОМ "АЛМАТЫ" (КАЗАХСТАН), 16 янв - РИА Новости. С аэродрома "Алматы" вылетел двадцатый самолет, который доставит в РФ российских миротворцев из состава коллективных сил ОДКБ, сообщили в Минобороны России.   Ранее сообщалось, что в воскресенье 19 самолетов ВКС с российскими миротворцами из состава сил ОДКБ прибыли из Казахстана на подмосковный аэродром "Чкаловский".   "Двадцатый самолет военно-транспортной авиации ВКС России, осуществляющий переброску возвращающихся из Казахстана российских миротворцев из состава Коллективных миротворческих сил ОДКБ, вылетел с аэродрома "Алматы", - говорится в сообщении.   Подчеркивается, что самолет Ан-124 "Руслан" летит на аэродром "Чкаловский" в Подмосковье. "Перед вылетом подразделения российских миротворцев на аэродроме "Алматы" в Республике Казахстан осуществили погрузку военной техники в самолеты военно-транспортной авиации ВКС России", - добавляется в сообщении.   Массов

## 3. Обзор полученных данных

In [148]:
news = []
with open('ria_news.json') as file:
    for line in file:
        news.append(json.loads(line))

In [149]:
data = pd.DataFrame(news)

In [150]:
len(data['article_id'].unique()), data.shape[0]

(7489, 7583)

Возможно, некоторые статьи пересекаются, и для таких статей нужно будет принять решение. Если они имеют более 1й категории, то не совсем понятно, что с ними делать. Если же категория одна и та же, то нужно просто удалить дубликаты.

In [151]:
data.drop(['tags', 'title', 'text'], axis=1)

,article_id,category
0,https://ria.ru/20220116/volki-1768107463.html,incidents
1,https://ria.ru/20220116/salyut-1768103772.html,incidents
2,https://ria.ru/20220116/dtp-1768101756.html,incidents
3,https://ria.ru/20220116/pozhar-1768079480.html,incidents
4,https://ria.ru/20220116/tsisterna-1768078676.html,incidents
...,...,...
7578,https://radiosputnik.ria.ru/20210810/tank-1745...,defense_safety
7579,https://ria.ru/20210810/tsirkon-1745190858.html,defense_safety
7580,https://ria.ru/20210810/tanki-1745189008.html,defense_safety
7581,https://ria.ru/20210810/avangard-1745187277.html,defense_safety


Как видно, нет повторений в одной категории.

In [152]:
alinks = data['article_id'].value_counts()
alinks[alinks > 1].to_frame()

,article_id
https://ria.ru/20220113/dolzhniki-1767738212.html,2
https://radiosputnik.ria.ru/20220115/dtp-1767978450.html,2
https://ria.ru/20211125/soyuz-1760642188.html,2
https://ria.ru/20220110/pereboi-1767173614.html,2
https://radiosputnik.ria.ru/20220111/lenoblast-1767310253.html,2
...,...
https://ria.ru/20211228/megafon-1765902629.html,2
https://ria.ru/20211229/kity-1766039795.html,2
https://radiosputnik.ria.ru/20211227/1765711626.html,2
https://ria.ru/20220114/kosulya-1767835405.html,2


Можно видеть, что есть повторяющиеся по 2 раза ссылки. Детальнее рассмотрим записи с такими ссылками.

In [153]:
data[data['article_id'] == 'https://ria.ru/20211230/vychet-1766198342.html']

,article_id,title,category,tags,text
1980,https://ria.ru/20211230/vychet-1766198342.html,Эксперт оценил идею ввести налоговый вычет при...,economy,"[Экономика, Общество, Авто, Институт исследова...","МОСКВА, 30 дек - РИА Новости. Предоставлять на..."
4326,https://ria.ru/20211230/vychet-1766198342.html,Эксперт оценил идею ввести налоговый вычет при...,society,"[Экономика, Общество, Авто, Институт исследова...","МОСКВА, 30 дек - РИА Новости. Предоставлять на..."


Такие ссылки можно удалить все, под предлогом того, что они имеют спорные классы. Либо же можно оставить по 1 из них.  Воспользуемся 1 способом.

In [154]:
data_fixed = data[~data['article_id'].isin(alinks[alinks > 1].index)]
data_fixed

,article_id,title,category,tags,text
0,https://ria.ru/20220116/volki-1768107463.html,Прокуратура проверяет сообщения о нападениях в...,incidents,"[Происшествия, Новгородская область]","С.-ПЕТЕРБУРГ, 16 янв - РИА Новости. Прокуратур..."
1,https://ria.ru/20220116/salyut-1768103772.html,В Минводах салют выбил окна в многоэтажке и по...,incidents,"[Происшествия, Минеральные Воды]","ПЯТИГОРСК, 16 янв - РИА Новости. Окна многоква..."
2,https://ria.ru/20220116/dtp-1768101756.html,В ДПТ под Красноярском погибли два ребенка и в...,incidents,"[Происшествия, Красноярск, Сосновоборск, Красн...","КРАСНОЯРСК, 16 янв - РИА Новости. Два малолетн..."
3,https://ria.ru/20220116/pozhar-1768079480.html,В Новосибирске потушили загоревшуюся цистерну ...,incidents,"[Происшествия, Новосибирская область, Октябрьс...","НОВОСИБИРСК, 16 янв – РИА Новости. Загоревшуюс..."
4,https://ria.ru/20220116/tsisterna-1768078676.html,В Новосибирске загорелась цистерна с бензином,incidents,"[Происшествия, Новосибирская область, Октябрьс...","НОВОСИБИРСК, 16 янв – РИА Новости. Цистерна с ..."
...,...,...,...,...,...
7578,https://radiosputnik.ria.ru/20210810/tank-1745...,Российская армия получит 20 новых танков Т-14 ...,defense_safety,"[В России, Безопасность, Технологии, Армата, А...","МОСКВА, 10 августа/ Радио Sputnik. В Вооруженн..."
7579,https://ria.ru/20210810/tsirkon-1745190858.html,"Испытания гиперзвуковой ракеты ""Циркон"" заверш...",defense_safety,"[Новое оружие России, Безопасность, Министерст...","МОСКВА, 10 авг - РИА Новости. Государственные ..."
7580,https://ria.ru/20210810/tanki-1745189008.html,Вооруженные силы до конца года получат 20 нове...,defense_safety,"[Новое оружие России, Безопасность, Министерст...","МОСКВА, 10 авг — РИА Новости. До конца года в ..."
7581,https://ria.ru/20210810/avangard-1745187277.html,"Второй полк комплекса ""Авангард"" поступит на в...",defense_safety,"[Новое оружие России, Безопасность, Сергей Шой...","МОСКВА, 10 авг - РИА Новости. Второй полк межк..."


In [155]:
data_fixed[data_fixed['article_id'].isin(alinks[alinks > 1].index)]

,article_id,title,category,tags,text


In [158]:
data_fixed['category'].value_counts()

incidents         1587
economy           1504
society           1476
defense_safety    1422
science           1406
Name: category, dtype: int64

Видим, что имеется больше 1000 уникальных статей каждй категории. Сохраним данные.

Также удалим записи, у которых пустая колонка текста.

In [159]:
data_fixed = data_fixed[data_fixed['text'].apply(len) > 0]
data_fixed

,article_id,title,category,tags,text
0,https://ria.ru/20220116/volki-1768107463.html,Прокуратура проверяет сообщения о нападениях в...,incidents,"[Происшествия, Новгородская область]","С.-ПЕТЕРБУРГ, 16 янв - РИА Новости. Прокуратур..."
1,https://ria.ru/20220116/salyut-1768103772.html,В Минводах салют выбил окна в многоэтажке и по...,incidents,"[Происшествия, Минеральные Воды]","ПЯТИГОРСК, 16 янв - РИА Новости. Окна многоква..."
2,https://ria.ru/20220116/dtp-1768101756.html,В ДПТ под Красноярском погибли два ребенка и в...,incidents,"[Происшествия, Красноярск, Сосновоборск, Красн...","КРАСНОЯРСК, 16 янв - РИА Новости. Два малолетн..."
3,https://ria.ru/20220116/pozhar-1768079480.html,В Новосибирске потушили загоревшуюся цистерну ...,incidents,"[Происшествия, Новосибирская область, Октябрьс...","НОВОСИБИРСК, 16 янв – РИА Новости. Загоревшуюс..."
4,https://ria.ru/20220116/tsisterna-1768078676.html,В Новосибирске загорелась цистерна с бензином,incidents,"[Происшествия, Новосибирская область, Октябрьс...","НОВОСИБИРСК, 16 янв – РИА Новости. Цистерна с ..."
...,...,...,...,...,...
7578,https://radiosputnik.ria.ru/20210810/tank-1745...,Российская армия получит 20 новых танков Т-14 ...,defense_safety,"[В России, Безопасность, Технологии, Армата, А...","МОСКВА, 10 августа/ Радио Sputnik. В Вооруженн..."
7579,https://ria.ru/20210810/tsirkon-1745190858.html,"Испытания гиперзвуковой ракеты ""Циркон"" заверш...",defense_safety,"[Новое оружие России, Безопасность, Министерст...","МОСКВА, 10 авг - РИА Новости. Государственные ..."
7580,https://ria.ru/20210810/tanki-1745189008.html,Вооруженные силы до конца года получат 20 нове...,defense_safety,"[Новое оружие России, Безопасность, Министерст...","МОСКВА, 10 авг — РИА Новости. До конца года в ..."
7581,https://ria.ru/20210810/avangard-1745187277.html,"Второй полк комплекса ""Авангард"" поступит на в...",defense_safety,"[Новое оружие России, Безопасность, Сергей Шой...","МОСКВА, 10 авг - РИА Новости. Второй полк межк..."


## 4. Конвертирование в JSON формат

In [160]:
js = json.loads(data_fixed.to_json(force_ascii=False, orient='records'))

In [161]:
with open('corpus.json', 'w') as file:
    for i in js:
        file.write(json.dumps(i) + '\n')